In [1]:
#import dependencies
import pandas as pd
import json
from flask import Flask, render_template,jsonify

# Imports the method used for connecting to DBs
from sqlalchemy import create_engine

# Imports the methods needed to abstract classes into tables
from sqlalchemy.ext.declarative import declarative_base

# Allow us to declare column types
from sqlalchemy import Column, Integer, String, Float ,Date

from sqlalchemy.orm import Session

from sqlalchemy import inspect
from sqlalchemy.ext.automap import automap_base

In [2]:
# ===========================Database Connection========================

# Create Database Connection
engine = create_engine("sqlite:///censusdata.sqlite", echo=False)
Base = automap_base()
Base.prepare(engine, reflect=True)
session = Session(engine)

conn = engine.connect()
#Base.metadata.create_all(conn)
session = Session(bind=engine)

In [3]:
aian = Base.classes.census_aian
asian = Base.classes.census_asian
black = Base.classes.census_black
hispanic = Base.classes.census_hispanic
mixed = Base.classes.census_mixed
other = Base.classes.census_other
whites = Base.classes.census_whites

In [4]:
# the queries
df_aian = pd.read_sql_query("select CountyID,CountyName,TotalPopulation,Year from census_aian;", conn)
df_aian["ID"]=df_aian["CountyID"]+df_aian["Year"]

df_aian=df_aian.rename(columns={'TotalPopulation': 'aian_population'})
df_aian.head()

,CountyID,CountyName,aian_population,Year,ID
0,2020,"Anchorage Municipality, Alaska",8091,2000,20202000
1,4001,"Apache County, Arizona",12081,2000,40012000
2,4005,"Coconino County, Arizona",8367,2000,40052000
3,4013,"Maricopa County, Arizona",20744,2000,40132000
4,4017,"Navajo County, Arizona",13271,2000,40172000


In [5]:
df_asian = pd.read_sql_query("select CountyID,CountyName,TotalPopulation,Year from census_asian;", conn)
df_asian["ID"]=df_asian["CountyID"]+df_asian["Year"]

df_asian = df_asian.rename(columns={'TotalPopulation': 'asian_population'})
df_asian.loc[df_asian['CountyID']=='1001']

,CountyID,CountyName,asian_population,Year,ID
0,1001,"Autauga County, Alabama",43,2000,10012000


In [6]:
df_black = pd.read_sql_query("select CountyID,CountyName,TotalPopulation,Year from census_black;", conn)
df_black["ID"]=df_black["CountyID"]+df_black["Year"]
df_black=df_black.rename(columns={'TotalPopulation': 'black_population'})
df_black.head()

,CountyID,CountyName,black_population,Year,ID
0,1001,"Autauga County, Alabama",2566,2000,10012000
1,1003,"Baldwin County, Alabama",4650,2000,10032000
2,1005,"Barbour County, Alabama",4446,2000,10052000
3,1007,"Bibb County, Alabama",1306,2000,10072000
4,1009,"Blount County, Alabama",146,2000,10092000


In [7]:
df_hispanic = pd.read_sql_query("select CountyID,CountyName,TotalPopulation,Year from census_hispanic;", conn)
df_hispanic["ID"]=df_hispanic["CountyID"]+df_hispanic["Year"]

df_hispanic=df_hispanic.rename(columns={'TotalPopulation': 'hispanic_population'})
df_hispanic.head()


,CountyID,CountyName,hispanic_population,Year,ID
0,1001,"Autauga County, Alabama",107,2000,10012000
1,1003,"Baldwin County, Alabama",605,2000,10032000
2,1005,"Barbour County, Alabama",103,2000,10052000
3,1007,"Bibb County, Alabama",43,2000,10072000
4,1009,"Blount County, Alabama",606,2000,10092000


In [8]:
df_mixed = pd.read_sql_query("select CountyID,CountyName,TotalPopulation,Year from census_mixed;", conn)
df_mixed["ID"]=df_mixed["CountyID"]+df_mixed["Year"]
df_mixed=df_mixed.rename(columns={'TotalPopulation': 'mixed_population'})
df_mixed.head()

,CountyID,CountyName,mixed_population,Year,ID
0,1001,"Autauga County, Alabama",171,2000,10012000
1,1003,"Baldwin County, Alabama",566,2000,10032000
2,1005,"Barbour County, Alabama",75,2000,10052000
3,1007,"Bibb County, Alabama",61,2000,10072000
4,1009,"Blount County, Alabama",183,2000,10092000


In [9]:
df_other = pd.read_sql_query("select CountyID,CountyName,TotalPopulation,Year from census_other;", conn)
df_other["ID"]=df_other["CountyID"]+df_other["Year"]
df_other=df_other.rename(columns={'TotalPopulation': 'other_population'})
df_other.head()

,CountyID,CountyName,other_population,Year,ID
0,1001,"Autauga County, Alabama",31,2000,10012000
1,1003,"Baldwin County, Alabama",212,2000,10032000
2,1005,"Barbour County, Alabama",57,2000,10052000
3,1007,"Bibb County, Alabama",15,2000,10072000
4,1009,"Blount County, Alabama",307,2000,10092000


In [10]:
df_whites = pd.read_sql_query("select CountyID,CountyName,TotalPopulation,Year from census_whites;", conn)
df_whites["ID"]=df_whites["CountyID"]+df_whites["Year"]
# df_whites=df_whites.set_index('ID')
df_whites=df_whites.rename(columns={'TotalPopulation': 'whites_population'})


In [30]:
result=pd.merge((pd.merge((pd.merge((pd.merge((pd.merge(pd.merge(df_whites,df_other,on='ID', how='outer'),df_mixed,on='ID', how='outer')),df_hispanic,on='ID', how='outer')),df_black,on='ID', how='outer')),df_asian,on='ID', how='outer')),df_aian,on='ID', how='outer')
result['Year'].unique()

array([nan, '2000', '2005', '2008', '2010', '2013', '2016'], dtype=object)

In [31]:
merge_data=result.drop(['CountyID_y','CountyName_y','Year_y','CountyID_x','CountyName_x','Year_x'], axis=1)

In [60]:
merge_data = merge_data.fillna(0)  
merge_data=merge_data.drop_duplicates()
merge_data.head()

,whites_population,ID,other_population,mixed_population,hispanic_population,black_population,asian_population,CountyID,CountyName,aian_population,Year,Total
0,13103.0,10012000,31.0,171.0,107.0,2566.0,43.0,0,0,0.0,0,16021.0
2,49458.0,10032000,212.0,566.0,605.0,4650.0,132.0,0,0,0.0,0,55623.0
4,5815.0,10052000,57.0,75.0,103.0,4446.0,3.0,0,0,0.0,0,10499.0
6,5978.0,10072000,15.0,61.0,43.0,1306.0,6.0,0,0,0.0,0,7409.0
8,18397.0,10092000,307.0,183.0,606.0,146.0,35.0,0,0,0.0,0,19674.0


In [67]:
merge_data['Total']=merge_data['other_population']+merge_data['mixed_population']+merge_data['hispanic_population']+merge_data['black_population'] + merge_data['asian_population']+merge_data['aian_population']+merge_data['whites_population']

In [68]:
merge_new=merge_data[merge_data.Year != 0]

In [69]:
merge_new.head()

,whites_population,ID,other_population,mixed_population,hispanic_population,black_population,asian_population,CountyID,CountyName,aian_population,Year,Total
138,74229.0,20202000,1618.0,3596.0,3889.0,5388.0,4155.0,2020,"Anchorage Municipality, Alaska",8091.0,2000,100966.0
188,5143.0,40012000,329.0,201.0,816.0,29.0,39.0,4001,"Apache County, Arizona",12081.0,2000,18638.0
192,29387.0,40052000,1311.0,585.0,3617.0,501.0,291.0,4005,"Coconino County, Arizona",8367.0,2000,44059.0
202,942545.0,40132000,87938.0,23455.0,186066.0,40573.0,21698.0,4013,"Maricopa County, Arizona",20744.0,2000,1323019.0
206,16674.0,40172000,943.0,487.0,2056.0,246.0,48.0,4017,"Navajo County, Arizona",13271.0,2000,33725.0


In [70]:
merge_new['%white']=(merge_new['whites_population']/merge_new['Total'])*100
merge_new['%black']=(merge_new['black_population']/merge_new['Total'])*100
merge_new['%mixed']=(merge_new['mixed_population']/merge_new['Total'])*100
merge_new['%others']=(merge_new['other_population']/merge_new['Total'])*100
merge_new['%asian']=(merge_new['asian_population']/merge_new['Total'])*100
merge_new['%aian']=(merge_new['aian_population']/merge_new['Total'])*100
merge_new['%hispanic']=(merge_new['hispanic_population']/merge_new['Total'])*100


/Users/sumana/anaconda/envs/PythonData/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/Users/sumana/anaconda/envs/PythonData/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/Users/sumana/anaconda/envs/PythonData/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_i

In [71]:
merge_new=merge_new.drop(['Total','other_population','mixed_population','hispanic_population','black_population','asian_population','aian_population','whites_population'], axis=1)

In [72]:
merge_new

,ID,CountyID,CountyName,Year,%white,%black,%mixed,%others,%asian,%aian,%hispanic
138,20202000,2020,"Anchorage Municipality, Alaska",2000,73.518808,5.336450,3.561595,1.602520,4.115247,8.013589,3.851792
188,40012000,4001,"Apache County, Arizona",2000,27.594162,0.155596,1.078442,1.765211,0.209250,64.819187,4.378152
192,40052000,4005,"Coconino County, Arizona",2000,66.699199,1.137112,1.327765,2.975556,0.660478,18.990445,8.209446
202,40132000,4013,"Maricopa County, Arizona",2000,71.241985,3.066698,1.772839,6.646768,1.640037,1.567929,14.063744
206,40172000,4017,"Navajo County, Arizona",2000,49.441067,0.729429,1.444033,2.796145,0.142328,39.350630,6.096368
208,40192000,4019,"Pima County, Arizona",2000,66.352851,2.252187,1.943213,7.810273,1.451809,2.679088,17.510580
404,60372000,6037,"Los Angeles County, California",2000,42.380297,8.344150,3.330040,12.365746,8.750076,0.350952,24.478740
426,60592000,6059,"Orange County, California",2000,61.837165,1.410392,2.657253,7.510470,9.836542,0.356720,16.391458
432,60652000,6065,"Riverside County, California",2000,58.847547,4.695755,2.583208,10.259612,2.415036,1.020529,20.178313
434,60672000,6067,"Sacramento County, California",2000,63.614878,8.287711,3.732617,4.904879,7.755439,1.033572,10.670904
